<a href="https://colab.research.google.com/github/rahulbhoyar1995/KaggleGPT-Research/blob/main/research/codebase/1_jupyter_files/8_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install python-dotenv

In [2]:
!ls -a

.  ..  .config	.env  .ipynb_checkpoints  knowlege_base_kaggle_datasets.pdf  sample_data


In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
pip install openai

In [6]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI

client = OpenAI()

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("-"*100)
        print("Citations :")
        print("\n".join(citations))


In [7]:
from openai import OpenAI


class LLMChatBot:
  def __init__(self):
      self.client = OpenAI()
      self.assistant = None
      self.vector_store = None
      self.thread = None
      self.run = None

  def creation_of_assistant(self,instructions_text):
      self.assistant = self.client.beta.assistants.create(
      name="Kaggle Based Recommendation System",
      instructions=instructions_text,
      model="gpt-4o",
      tools=[{"type": "file_search"}],)


  def creation_of_vector_store(self,pdf_file_path):
      self.vector_store = self.client.beta.vector_stores.create(name="Financial Statements")
      # Ready the files for upload to OpenAI
      file_paths = [pdf_file_path] # You can add multiple files here
      file_streams = [open(path, "rb") for path in file_paths]

      # Use the upload and poll SDK helper to upload the files, add them to the vector store,
      # and poll the status of the file batch for completion.
      file_batch = self.client.beta.vector_stores.file_batches.upload_and_poll(vector_store_id=self.vector_store.id, files=file_streams)


  def updating_assistant(self):
      self.assistant = self.client.beta.assistants.update(
      assistant_id = self.assistant.id,
      tool_resources={"file_search": {"vector_store_ids": [self.vector_store.id]}},)


  def creation_of_thread(self):
      self.thread = self.client.beta.threads.create()


  def response(self,query, streaming = False):
      message = self.client.beta.threads.messages.create(
      thread_id=self.thread.id,
      role="user",
      content=query)

      if not streaming:
          # Without streaming
          run = client.beta.threads.runs.create_and_poll(thread_id= self.thread.id, assistant_id= self.assistant.id)
          messages = list(client.beta.threads.messages.list(thread_id=self.thread.id, run_id=run.id))

          message_content = messages[0].content[0].text
          annotations = message_content.annotations
          citations = []
          for index, annotation in enumerate(annotations):
              message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
              if file_citation := getattr(annotation, "file_citation", None):
                  cited_file = client.files.retrieve(file_citation.file_id)
                  citations.append(f"[{index}] {cited_file.filename}")

          #print(message_content.value)
          print("-"*100)
          print("Citations :")
          print("\n".join(citations))
          citations_text = "\n".join(citations)
          return message_content.value , citations_text


      #With streaming
      with self.client.beta.threads.runs.stream(
          thread_id= self.thread.id,
          assistant_id= self.assistant.id,
          instructions="You are Kaggle Dataset Recommender system.",
          event_handler=EventHandler(),) as stream:
          stream.until_done()


In [8]:
instructions = """
Project Exposé must be in computer science, machine learning, and artificial intelligence in general. Students might not give a precise description and ideas. They just want to have any proposed topics with datasets.
        Your tasks are as follows:
        1. You should provide at least 10 different datasets on computer vision, natural language processing, or time series.
        2. You should display results in a table for easy viewing.
        3. You should group datasets by topic.
        4. The response should have the Kaggle Datasets link.

        You identify the input language and give the response in the same language, However, you should provide the dataset's information in English.
"""

In [9]:
llm_obj = LLMChatBot()
llm_obj.creation_of_assistant(instructions)
llm_obj.creation_of_vector_store("knowlege_base_kaggle_datasets.pdf")
llm_obj.creation_of_thread()
llm_obj.updating_assistant()

In [10]:
query ="""

Hope you are doing well. Myself Saddam Rafiq. I am your student of Deep Learning. First of all, I would like to share that my learning experience with you are very helpful, the way you taught us is unique and meaningful.I am writing to express my interest in the possibility of working on my master's thesis under your supervision.I want you to be my supervisor so I can perform well in my thesis.I will be waiting for your kind response.

"""

## CASE 1: WITHOUT STREAMING

In [11]:
response_1,citations_text = llm_obj.response(query)


----------------------------------------------------------------------------------------------------
Citations :



In [12]:
response_1

"Hello Saddam Rafiq,\n\nThank you for reaching out and sharing your positive experience in my Deep Learning course. I'm glad to hear that you found the teaching methods effective and meaningful.\n\nFor your master's thesis, I'd be happy to discuss potential topics you could explore under my supervision. Below, I have proposed several datasets related to computer vision, natural language processing, and time series which you might find interesting for your research:\n\n| **Topic**               | **Dataset**                                                                                     |                                                                 **Kaggle Link** |\n|-------------------------|-------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------|\n| **Computer Vision**     | Intel Image Classification                                                          

Hello Saddam Rafiq,

Thank you for reaching out and sharing your positive experience in my Deep Learning course. I'm glad to hear that you found the teaching methods effective and meaningful.

For your master's thesis, I'd be happy to discuss potential topics you could explore under my supervision. Below, I have proposed several datasets related to computer vision, natural language processing, and time series which you might find interesting for your research:

| **Topic**               | **Dataset**                                                                                     |                                                                 **Kaggle Link** |
|-------------------------|-------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------|
| **Computer Vision**     | Intel Image Classification                                                                      | [Intel Image Classification](https://www.kaggle.com/puneet6060/intel-image-classification)        |
| **Computer Vision**     | COVID-19 Radiography Database                                                                   | [COVID-19 Radiography Database](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database)|
| **Computer Vision**     | Plant Seedlings Classification                                                                  | [Plant Seedlings Classification](https://www.kaggle.com/c/plant-seedlings-classification)          |
| **Computer Vision**     | Street View House Numbers                                                                       | [Street View House Numbers](https://www.kaggle.com/competitions/svhn-cropped)                      |
| **Natural Language Processing (NLP)** | Twitter Sentiment Analysis                                                         | [Twitter Sentiment Analysis](https://www.kaggle.com/kazanova/sentiment140)                         |
| **Natural Language Processing (NLP)** | 20 Newsgroups                                                                      | [20 Newsgroups](https://www.kaggle.com/c/20-newsgroups-classifier/data)                            |
| **Natural Language Processing (NLP)** | IMDb Movie Reviews                                                                 | [IMDb Movie Reviews](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)     |
| **Natural Language Processing (NLP)** | Quora Question Pairs                                                               | [Quora Question Pairs](https://www.kaggle.com/c/quora-question-pairs)                              |
| **Time Series**         | UCI Electricity Load                                                                           | [UCI Electricity Load Forecasting](https://www.kaggle.com/syuryu/forecasting-electricity-loadsj)   |
| **Time Series**         | Stocks Data                                                                                     | [Stocks Data](https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs)     |
| **Time Series**         | Household Power Consumption                                                                    | [Household Power Consumption](https://www.kaggle.com/uciml/electric-power-consumption-data-set)    |

Please take your time to review these datasets and see if any align with your interests. Feel free to propose any other topics or datasets you have in mind. Once you've selected a topic or dataset, we can develop a more detailed plan for your thesis.

Looking forward to your response.

Best regards,

In [13]:
citations_text

''

In [14]:
response_2,citations_text_2 = llm_obj.response("What can you tell me about these datasets ?")


----------------------------------------------------------------------------------------------------
Citations :



In [15]:
response_2

'Certainly! Below is a brief description of each dataset listed, grouped by topic:\n\n### Computer Vision\n\n1. **Intel Image Classification**\n   - **Description**: This dataset contains images of natural scenes categorized into six classes: buildings, forest, glacier, mountain, sea, and street.\n   - **Link**: [Intel Image Classification](https://www.kaggle.com/puneet6060/intel-image-classification)\n\n2. **COVID-19 Radiography Database**\n   - **Description**: This dataset contains X-ray images of patients with COVID-19, normal, and viral pneumonia. It can be used for the early detection of COVID-19 using radiographs.\n   - **Link**: [COVID-19 Radiography Database](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database)\n\n3. **Plant Seedlings Classification**\n   - **Description**: The dataset contains images of 12 different species of plant seedlings at various stages of growth. The objective is to classify the plant species.\n   - **Link**: [Plant Seedlings Classifica

Certainly! Below is a brief description of each dataset listed, grouped by topic:

### Computer Vision

1. **Intel Image Classification**
   - **Description**: This dataset contains images of natural scenes categorized into six classes: buildings, forest, glacier, mountain, sea, and street.
   - **Link**: [Intel Image Classification](https://www.kaggle.com/puneet6060/intel-image-classification)

2. **COVID-19 Radiography Database**
   - **Description**: This dataset contains X-ray images of patients with COVID-19, normal, and viral pneumonia. It can be used for the early detection of COVID-19 using radiographs.
   - **Link**: [COVID-19 Radiography Database](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database)

3. **Plant Seedlings Classification**
   - **Description**: The dataset contains images of 12 different species of plant seedlings at various stages of growth. The objective is to classify the plant species.
   - **Link**: [Plant Seedlings Classification](https://www.kaggle.com/c/plant-seedlings-classification)

4. **Street View House Numbers (SVHN)**
   - **Description**: This dataset contains cropped images of digits from street view images. It is used for recognizing digits in natural scene images.
   - **Link**: [Street View House Numbers](https://www.kaggle.com/competitions/svhn-cropped)

### Natural Language Processing (NLP)

5. **Twitter Sentiment Analysis (Sentiment140)**
   - **Description**: This dataset contains 1.6 million tweets with labels (positive, negative, neutral). It can be used to train models for sentiment analysis.
   - **Link**: [Twitter Sentiment Analysis](https://www.kaggle.com/kazanova/sentiment140)

6. **20 Newsgroups**
   - **Description**: The dataset contains approximately 20,000 newsgroup documents spread across 20 different newsgroups. It is useful for text classification tasks.
   - **Link**: [20 Newsgroups](https://www.kaggle.com/c/20-newsgroups-classifier/data)

7. **IMDb Movie Reviews**
   - **Description**: This dataset contains 50,000 movie reviews, categorized into positive and negative reviews. It is commonly used for sentiment analysis.
   - **Link**: [IMDb Movie Reviews](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

8. **Quora Question Pairs**
   - **Description**: This dataset consists of question pairs from Quora, labeled as duplicate (if they are the same question) or not duplicate, making it useful for semantic similarity tasks.
   - **Link**: [Quora Question Pairs](https://www.kaggle.com/c/quora-question-pairs)

### Time Series

9. **UCI Electricity Load Forecasting**
   - **Description**: This dataset features electricity consumption data, suitable for forecasting and time-series analysis tasks related to energy consumption.
   - **Link**: [UCI Electricity Load Forecast](https://www.kaggle.com/syuryu/forecasting-electricity-loadsj)

10. **Stocks Data**
    - **Description**: This dataset includes historical price and volume data for all US stocks and ETFs. It can be used for financial market analysis and stock price prediction.
    - **Link**: [Stocks Data](https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs)

11. **Household Power Consumption**
    - **Description**: The dataset contains measurements of electrical power consumption in one household over a period of four years. It is useful for time-series forecasting.
    - **Link**: [Household Power Consumption](https://www.kaggle.com/uciml/electric-power-consumption-data-set)

These datasets provide a range of options across different domains of machine learning. Depending on your interests, you can choose one of these datasets to explore further in your thesis. If you have specific questions about any dataset or need guidance on a particular topic, feel free to ask.

In [16]:
citations_text_2

''

In [17]:
response_3 = llm_obj.response("What is the working title of my thesis?")


----------------------------------------------------------------------------------------------------
Citations :



In [18]:
response_3[0]

'Choosing a working title for your thesis depends on the specific dataset and research question you decide to pursue. Here are some example working titles for various topics and datasets:\n\n### Computer Vision\n\n1. **Intel Image Classification**\n   - **Working Title**: "Image Classification of Natural Scenes Using Deep Learning Techniques on the Intel Image Dataset"\n   \n2. **COVID-19 Radiography Database**\n   - **Working Title**: "Automated Detection of COVID-19 from X-ray Images Using Convolutional Neural Networks"\n\n3. **Plant Seedlings Classification**\n   - **Working Title**: "Species Classification of Plant Seedlings Using Deep Learning Approaches"\n\n4. **Street View House Numbers (SVHN)**\n   - **Working Title**: "Digit Recognition in Natural Scene Images Using Deep Neural Networks"\n\n### Natural Language Processing (NLP)\n\n5. **Twitter Sentiment Analysis (Sentiment140)**\n   - **Working Title**: "Sentiment Analysis of Twitter Posts Using Natural Language Processing and

Choosing a working title for your thesis depends on the specific dataset and research question you decide to pursue. Here are some example working titles for various topics and datasets:

### Computer Vision

1. **Intel Image Classification**
   - **Working Title**: "Image Classification of Natural Scenes Using Deep Learning Techniques on the Intel Image Dataset"
   
2. **COVID-19 Radiography Database**
   - **Working Title**: "Automated Detection of COVID-19 from X-ray Images Using Convolutional Neural Networks"

3. **Plant Seedlings Classification**
   - **Working Title**: "Species Classification of Plant Seedlings Using Deep Learning Approaches"

4. **Street View House Numbers (SVHN)**
   - **Working Title**: "Digit Recognition in Natural Scene Images Using Deep Neural Networks"

### Natural Language Processing (NLP)

5. **Twitter Sentiment Analysis (Sentiment140)**
   - **Working Title**: "Sentiment Analysis of Twitter Posts Using Natural Language Processing and Machine Learning"

6. **20 Newsgroups**
   - **Working Title**: "Text Classification of Newsgroup Documents Using Machine Learning Techniques"

7. **IMDb Movie Reviews**
   - **Working Title**: "Sentiment Analysis of Movie Reviews Using Deep Learning Approaches"

8. **Quora Question Pairs**
   - **Working Title**: "Semantic Similarity Detection in Quora Question Pairs Using Natural Language Processing"

### Time Series

9. **UCI Electricity Load Forecasting**
   - **Working Title**: "Time Series Analysis and Forecasting of Electricity Load Data Using Machine Learning Models"

10. **Stocks Data**
    - **Working Title**: "Predictive Modeling of Stock Prices Using Time-Series Data and Machine Learning"

11. **Household Power Consumption**
    - **Working Title**: "Forecasting Household Electrical Power Consumption Using Time-Series Analysis Techniques"

You can refine your working title based on the specific problem statement, objectives, and methods you plan to use in your research. Once you've decided on a specific dataset and research focus, we can further tailor the title to better fit your thesis.

In [19]:
response_4 = llm_obj.response("Summurise the extract of my text and your recommended dataset. Also tell me reason why you have suggested me particular datasets.")

----------------------------------------------------------------------------------------------------
Citations :



In [20]:
response_4[0]

"### Summary of Your Text\nYou, Saddam Rafiq, are expressing your interest in working on your master's thesis under the supervision of your Deep Learning instructor. You mention that your learning experience in the course has been helpful and effective, and you are eager to perform well in your thesis with the instructor's guidance. You are seeking recommendations for potential thesis topics and datasets to work with.\n\n### Recommended Datasets and Reasons\nThe suggested datasets are grouped by three main topics: computer vision, natural language processing (NLP), and time series. Here's a summary of the recommended datasets and the reasons for their selection:\n\n#### Computer Vision\n1. **Intel Image Classification**\n   - **Description**: Dataset containing images of six natural scene categories.\n   - **Reason**: Excellent for exploring image classification techniques using deep neural networks.\n\n2. **COVID-19 Radiography Database**\n   - **Description**: Contains X-ray images f

### Summary of Your Text
You, Saddam Rafiq, are expressing your interest in working on your master's thesis under the supervision of your Deep Learning instructor. You mention that your learning experience in the course has been helpful and effective, and you are eager to perform well in your thesis with the instructor's guidance. You are seeking recommendations for potential thesis topics and datasets to work with.

### Recommended Datasets and Reasons
The suggested datasets are grouped by three main topics: computer vision, natural language processing (NLP), and time series. Here's a summary of the recommended datasets and the reasons for their selection:

#### Computer Vision
1. **Intel Image Classification**
   - **Description**: Dataset containing images of six natural scene categories.
   - **Reason**: Excellent for exploring image classification techniques using deep neural networks.

2. **COVID-19 Radiography Database**
   - **Description**: Contains X-ray images for the detection of COVID-19.
   - **Reason**: Timely and impactful research topic, suitable for medical imaging and healthcare applications.

3. **Plant Seedlings Classification**
   - **Description**: Images of different plant species at various growth stages.
   - **Reason**: Relevant for agricultural applications and species classification.

4. **Street View House Numbers (SVHN)**
   - **Description**: Cropped images of digits from street views.
   - **Reason**: Popular benchmark dataset for digit recognition in natural images, good for OCR (Optical Character Recognition) tasks.

#### Natural Language Processing (NLP)
5. **Twitter Sentiment Analysis (Sentiment140)**
   - **Description**: Dataset of tweets labeled for sentiment analysis.
   - **Reason**: Ideal for sentiment analysis and opinion mining in social media data.

6. **20 Newsgroups**
   - **Description**: Documents from 20 different newsgroups.
   - **Reason**: Classic dataset for text classification and clustering tasks.

7. **IMDb Movie Reviews**
   - **Description**: Movie reviews labeled as positive or negative.
   - **Reason**: Widely used for sentiment analysis tasks in textual data.

8. **Quora Question Pairs**
   - **Description**: Pairs of questions labeled as duplicate or not.
   - **Reason**: Suitable for tasks involving semantic similarity and question answering.

#### Time Series
9. **UCI Electricity Load Forecasting**
   - **Description**: Electricity consumption data for forecasting.
   - **Reason**: Relevant for energy management and load forecasting applications.

10. **Stocks Data**
    - **Description**: Historical stock price and volume data.
    - **Reason**: Useful for financial analysis and predictive modeling in stock markets.

11. **Household Power Consumption**
    - **Description**: Electrical power consumption measurements in a household.
    - **Reason**: Practical application in energy consumption forecasting and smart home management.

### Reason for Dataset Suggestions
- **Variety of Applications**: These datasets span across key areas such as healthcare, agriculture, finance, and energy, providing you with the option to choose a topic that interests you the most.
- **Strong Real-World Relevance**: Each dataset corresponds to real-world problems, ensuring that your research has practical implications and social impact.
- **Popularity and Benchmarking**: Many of these datasets are popular and well-benchmarked, offering a good foundation and comparison for your results.
- **Diverse Techniques**: These datasets allow for the exploration of diverse machine learning techniques, including deep learning for image and text data, and time-series analysis for forecasting.

These reasons collectively aim to provide a robust and impactful start to your master's thesis, facilitating meaningful research and potential contributions to the field.

In [21]:
response_5 = llm_obj.response("Can you please tell me the sources of your recommendations ? Is there any file from which you have got this information about datasets ?")

----------------------------------------------------------------------------------------------------
Citations :



In [22]:
response_5[0]

'The information about the datasets and recommendations provided earlier is sourced from publicly accessible data repositories, specifically from Kaggle, a well-known platform for datasets and data science competitions.\n\n### Sources of Dataset Information\n1. **Intel Image Classification**\n   - **Source**: [Intel Image Classification on Kaggle](https://www.kaggle.com/puneet6060/intel-image-classification)\n\n2. **COVID-19 Radiography Database**\n   - **Source**: [COVID-19 Radiography Database on Kaggle](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database)\n\n3. **Plant Seedlings Classification**\n   - **Source**: [Plant Seedlings Classification on Kaggle](https://www.kaggle.com/c/plant-seedlings-classification)\n\n4. **Street View House Numbers (SVHN)**\n   - **Source**: [Street View House Numbers on Kaggle](https://www.kaggle.com/competitions/svhn-cropped)\n\n5. **Twitter Sentiment Analysis (Sentiment140)**\n   - **Source**: [Twitter Sentiment Analysis on Kaggle](htt

The information about the datasets and recommendations provided earlier is sourced from publicly accessible data repositories, specifically from Kaggle, a well-known platform for datasets and data science competitions.

### Sources of Dataset Information
1. **Intel Image Classification**
   - **Source**: [Intel Image Classification on Kaggle](https://www.kaggle.com/puneet6060/intel-image-classification)

2. **COVID-19 Radiography Database**
   - **Source**: [COVID-19 Radiography Database on Kaggle](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database)

3. **Plant Seedlings Classification**
   - **Source**: [Plant Seedlings Classification on Kaggle](https://www.kaggle.com/c/plant-seedlings-classification)

4. **Street View House Numbers (SVHN)**
   - **Source**: [Street View House Numbers on Kaggle](https://www.kaggle.com/competitions/svhn-cropped)

5. **Twitter Sentiment Analysis (Sentiment140)**
   - **Source**: [Twitter Sentiment Analysis on Kaggle](https://www.kaggle.com/kazanova/sentiment140)

6. **20 Newsgroups**
   - **Source**: [20 Newsgroups on Kaggle](https://www.kaggle.com/c/20-newsgroups-classifier/data)

7. **IMDb Movie Reviews**
   - **Source**: [IMDb Movie Reviews on Kaggle](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

8. **Quora Question Pairs**
   - **Source**: [Quora Question Pairs on Kaggle](https://www.kaggle.com/c/quora-question-pairs)

9. **UCI Electricity Load Forecasting**
   - **Source**: [UCI Electricity Load Forecasting on Kaggle](https://www.kaggle.com/syuryu/forecasting-electricity-loadsj)

10. **Stocks Data**
    - **Source**: [Stocks Data on Kaggle](https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs)

11. **Household Power Consumption**
    - **Source**: [Household Power Consumption on Kaggle](https://www.kaggle.com/uciml/electric-power-consumption-data-set)

No specific file from the uploaded documents was used to gather this dataset information; instead, it was based on my understanding of publicly available resources on the Kaggle platform, a common repository for data scientists and learners for accessing quality datasets for various domains of machine learning.

In [23]:
response_6 = llm_obj.response("Which document you have used to take this data from ? What is the context file here ?")

----------------------------------------------------------------------------------------------------
Citations :
[0] knowlege_base_kaggle_datasets.pdf
[1] knowlege_base_kaggle_datasets.pdf
[2] knowlege_base_kaggle_datasets.pdf
[3] knowlege_base_kaggle_datasets.pdf
[4] knowlege_base_kaggle_datasets.pdf
[5] knowlege_base_kaggle_datasets.pdf
[6] knowlege_base_kaggle_datasets.pdf
[7] knowlege_base_kaggle_datasets.pdf
[8] knowlege_base_kaggle_datasets.pdf
[9] knowlege_base_kaggle_datasets.pdf
[10] knowlege_base_kaggle_datasets.pdf


In [31]:
response_6[0]

"### Summary of Your Text\nYou, Saddam Rafiq, expressed interest in working on your master's thesis under the supervision of your Deep Learning instructor. You highlighted the effectiveness of the teaching methods in the course and are eager to excel in your thesis with the instructor's guidance. You are seeking recommendations for potential thesis topics and relevant datasets.\n\n### Recommended Datasets and Reasons\nThe recommended datasets span across three main areas: computer vision, natural language processing (NLP), and time series. Below are brief descriptions and reasons for each dataset:\n\n#### Computer Vision\n1. **Intel Image Classification**\n   - **Description**: Images of natural scenes in six categories.\n   - **Reason**: Ideal for image classification projects using deep neural networks.\n   - **Source**: [Intel Image Classification on Kaggle](https://www.kaggle.com/puneet6060/intel-image-classification)[0]\n\n2. **COVID-19 Radiography Database**\n   - **Description**

In [30]:
response_6[1]

'[0] knowlege_base_kaggle_datasets.pdf\n[1] knowlege_base_kaggle_datasets.pdf\n[2] knowlege_base_kaggle_datasets.pdf\n[3] knowlege_base_kaggle_datasets.pdf\n[4] knowlege_base_kaggle_datasets.pdf\n[5] knowlege_base_kaggle_datasets.pdf\n[6] knowlege_base_kaggle_datasets.pdf\n[7] knowlege_base_kaggle_datasets.pdf\n[8] knowlege_base_kaggle_datasets.pdf\n[9] knowlege_base_kaggle_datasets.pdf\n[10] knowlege_base_kaggle_datasets.pdf'

[0] knowlege_base_kaggle_datasets.pdf
[1] knowlege_base_kaggle_datasets.pdf
[2] knowlege_base_kaggle_datasets.pdf
[3] knowlege_base_kaggle_datasets.pdf
[4] knowlege_base_kaggle_datasets.pdf
[5] knowlege_base_kaggle_datasets.pdf
[6] knowlege_base_kaggle_datasets.pdf
[7] knowlege_base_kaggle_datasets.pdf
[8] knowlege_base_kaggle_datasets.pdf
[9] knowlege_base_kaggle_datasets.pdf
[10] knowlege_base_kaggle_datasets.pdf

## CASE 2: WITH STREAMING

In [25]:
instructions = """
Project Exposé must be in computer science, machine learning, and artificial intelligence in general. Students might not give a precise description and ideas. They just want to have any proposed topics with datasets.
        Your tasks are as follows:
        1. You should provide at least 10 different datasets on computer vision, natural language processing, or time series.
        2. You should display results in a table for easy viewing.
        3. You should group datasets by topic.
        4. The response should have the Kaggle Datasets link.

        You identify the input language and give the response in the same language, However, you should provide the dataset's information in English.
"""

In [26]:
query ="""

My name is Akashdeep Singh and I am a student of MS in Data Science.I am currently working on my thesis proposal. In my thesis, I would like to explore Improvement in process optimization through application of data analytics: A case of Amazon. As this topic falls within your field of expertise, I was wondering if you’d bewilling to supervise my thesis.I attached my draft thesis proposal to this email. Please let me know if you have any questions about the topic or process.I am looking forward to hearing from you.
"""

In [27]:
llm_obj_2 = LLMChatBot()
llm_obj_2.creation_of_assistant(instructions)
llm_obj_2.creation_of_vector_store("knowlege_base_kaggle_datasets.pdf")
llm_obj_2.creation_of_thread()
llm_obj_2.updating_assistant()

In [28]:
response_11 = llm_obj_2.response(query, streaming = True)


assistant > file_search


assistant > To explore improvements in process optimization at Amazon through data analytics for your thesis proposal, you might find the following datasets on Kaggle particularly useful:

1. [Amazon Best Sellers](https://www.kaggle.com/datasets/edwardtoledolpez/amazon-mexico-top-50-best-sellers) - This dataset includes top 50 best sellers on Amazon Mexico and Brazil, which can be used for sales and trend analysis .

2. [Amazon Business Research Analyst Dataset](https://www.kaggle.com/datasets/vikramxd/amazon-business-research-analyst-dataset) - This dataset is tailored for business research and can help in extracting significant insights for optimization and analytics projects at Amazon .

3. [Amazon US Customer Reviews Dataset](https://www.kaggle.com/datasets/cynthiarempel/amazon-us-customer-reviews-dataset) - A large dataset containing customer reviews, which can provide valuable insights into customer satisfaction and areas for process improvement .

Thes

In [32]:
response_12 = llm_obj_2.response("What is the context document from which you have provided me this data ?", streaming = True)


assistant > file_search


assistant > The dataset recommendations provided are derived from the "knowlege_base_kaggle_datasets.pdf" document, which lists various datasets available on Kaggle that can be relevant for data analysis and optimization projects. Specifically:

1. **Amazon Best Sellers** dataset can be used to analyze sales trends and performance of popular products on Amazon Mexico, helping in sales and inventory optimization .

2. **Amazon Business Research Analyst Dataset** is designed for business research, which includes data relevant for analyzing business decisions and operational processes at Amazon, useful for making process improvements through data analytics .

3. **Amazon US Customer Reviews Dataset** contains a large collection of customer reviews, useful for sentiment analysis to identify areas of customer satisfaction and dissatisfaction, and subsequently optimize customer service and product offerings .
--------------------------------------------------------